# Fine-tuning BERT with LoRA
---

**Objective:** In this practical exercise, we will the LoRA fine-tuning approach. Our main objective is to fine-tune a BERT model to categorize movie ratings binarily. The dataset consists of polarized ratings between positive and negative reviews and our goal is to increase the model's accuracy with fine-tuning



## Importing Needed Packages

### Prerequisite Libraries:

1. **transformers**: Hugging Face transformers library, which is used for working with pre-trained transformer-based models, such as BERT.
2. **peft**: Library that allows the usage of PEFT techniques such as LoRA.
3. **evaluate**: A library that makes evaluating and comparing models and reporting their performance easier and more standardized.

In [1]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Dataset Loading

The selected dataset for this Notebook is the **IMDB** dataset. The decision to use this dataset is based on two reasons.
- First, the dataset offers a substantial amount of data, providing a rich resource for training machine learning models effectively.
- Second, the dataset's straightforward columns are well-suited for the purpose of comparing various machine learning algorithms.


Let's start by downlaoding and loading the dataset:

In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Data Preprocessing

### Tokenization
Tokenization will allow us to feed batches of sequences at the same time:
- SMS need to be padded to the same length
- SMS need to be truncated to match the model's maximum input length.

To perform the tokenization of the data, we also need to choose a pre-trained tokenizer. For our case, the basic model (bert-base-cased) will be sufficient:

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Splitting dataset into Train and Test Sets
It is important to split the dataset into train and test sets while preserving the distribution of spam and ham texts. In our case this is already done with the previously set Tokenizer, so we are only going to fetch the train and test splits, and set the same seed for both so it's reproducible.

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Defining LoRA Configurations
Since we are using the peft library, we can set a LoRA configuration to be loaded later on, without much boiler-plate code.

r=  the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.

lora_alpha= responsible for the scaling of the weight matrices

lora_dropout= dropout techniques that avoids overfitting

In [5]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

### Loading the pre-trained BERT
Now we will load the pre-trained BERT, specifically for Sequence Classification, and set the number of labels to 2 since this is our goal

In [6]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Adding LoRA to our Model
To add the Lora configurations we set earlier to our model, we use the get_peft_model function, passing to it our model and the lora_config variable we set earlier.

In [7]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

## Creating our Metrics Function
We will create a compute_metrics function during this section that will be used during Training. We will evaluate our model based on accuracy, and this function will be passed onto our Trainer( ) class that will train our model with the LoRA configurations.

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Training our Model with LoRA
Finally, we will train our model and see how the accuracy enhances through each epoch. The Trainer( ) class will allow seamless implementation of our training process, by just adding the previous variables and functions we created earlier

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=25,)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 